<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/structured_30_D_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

def make_gaussian_data(n_samples=6000, n_features=30, n_classes=10, std=0.6, seed=SEED):
    rng = np.random.default_rng(seed)
    # Create class centers in feature space
    centers = rng.normal(loc=0.0, scale=2.0, size=(n_classes, n_features))
    # Balanced classes
    per_class = n_samples // n_classes
    xs, ys = [], []
    for c in range(n_classes):
        x_c = centers[c] + rng.normal(0, std, size=(per_class, n_features))
        y_c = np.full((per_class,), c, dtype=np.int32)
        xs.append(x_c); ys.append(y_c)
    X = np.vstack(xs).astype("float32")
    y = np.concatenate(ys).astype("int32")
    # Shuffle
    idx = rng.permutation(len(X))
    return X[idx], y[idx]

# Data
X, y = make_gaussian_data(n_samples=6000, n_features=30, n_classes=10, std=0.7)
n = len(X)
n_train = int(0.7 * n)
n_val = int(0.15 * n)
X_train, y_train = X[:n_train], y[:n_train]
X_val, y_val = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
X_test, y_test = X[n_train+n_val:], y[n_train+n_val:]

# tf.data pipelines
batch_size = 128
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(4096, seed=SEED).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Model (your style, with a touch of regularization)
class SpacetimeAI(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.curvature_layer = tf.keras.layers.Dense(512, activation='tanh')
        self.geodesic_layer  = tf.keras.layers.Dense(1024, activation='relu')
        self.dropout         = tf.keras.layers.Dropout(0.2)
        self.output_layer    = tf.keras.layers.Dense(10, activation='softmax')
    def call(self, inputs, training=False):
        x = self.curvature_layer(inputs)
        x = self.geodesic_layer(x)
        x = self.dropout(x, training=training)
        return self.output_layer(x)

model = SpacetimeAI()
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
os.makedirs("results", exist_ok=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=8, mode="max", restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5),
    tf.keras.callbacks.CSVLogger("results/training_history.csv")
]

history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=callbacks, verbose=1)

# Evaluate
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

# Save model
model.save("results/spacetime_ai.keras")
print("✅ Training history saved to results/training_history.csv")
print("✅ Model saved to results/spacetime_ai.keras")